#### Задача 1 (10 баллов). 

Представим себе, что мы хотим открыть музыкальный магазин и заведовать всеми его делами. Напишем программу, которая будет визуализировать наши мечты. Наверное, в первую очередь нам понадобится класс для самого магазина. У магазина есть счет, склад, возможность продавать инструменты и покупать их (может быть, что-то еще - что придумаете). Потом, нужен, конечно, класс для музыкального инструмента. Попробуйте реализовать музыкальные инструменты с помощью наследования: можно создать некий абстрактный класс "инструмент", а от него будут наследовать классы "гитара", "пианино" и так далее - при этом у гитар и пианино могут быть свои разные характеристики, н-р, у гитары количество струн, тип порожка и так далее. Можно и более подробную иерархию, если будете успевать (например, инструменты бывают электрическими и акустическими). Наконец, можно создать класс "покупатель", у которого будет кошелек. 

Наконец, неплохо бы не забыть написать некий код, который будет приводить наши классы в движение: имитировать рабочий день магазина (можно в цикле), с рандомной вероятностью посещение покупателей, давать нашему владельцу-пользователю возможность решить, не пора ли закупить новые инструменты или, может быть, затеять рекламную кампанию. 

Фантазия всячески поощряется, могу за нее поставить дополнительные баллы. 

In [56]:
import random


class MusicStore:
    def __init__(self, name, balance):
        self.name = name
        self.balance = balance
        self.inventory = {}
        self.promo_days = 0
        self.total_sold = 0
    
    def buy_instrument(self, instrument, quantity=1):
        cost = instrument.buy_price * quantity
        if cost > self.balance:
            print('Недостаточно средств для покупки инструмента :(')
        else:
            self.balance -= cost
            if instrument.name in self.inventory:
                self.inventory[instrument.name] += quantity
            else:
                self.inventory[instrument.name] = quantity
            print(f'Вы приобрели инструмент {instrument.name} за {cost} нотакоинов.') # нотакоины - игровая валюта
    
    def sell_instrument(self, instrument, quantity=1):
        if instrument.name in self.inventory and self.inventory[instrument.name] >= quantity:
            self.inventory[instrument.name] -= quantity
            total_price = instrument.sell_price * quantity
            self.balance += total_price
            self.total_sold += quantity
            return True
        else:
            return False
    
    def start_promotion(self, cost, effect_duration=3):
        if self.balance >= cost:
            self.balance -= cost
            self.promo_days = effect_duration
            print(f'Вы опубликовали рекламный пост у инфлюенсера за {cost} нотакоинов. Это привлечет больше клиентов в следующие {effect_duration} дней.')
        else:
            print(f'Недостаточно средств для рекламной кампании')
    
    def display_inventory(self):
        print(f'\nТекущий инвентарь магазина {self.name}:')
        if not self.inventory:
            print('Склад пуст')
        else:
            for instrument, quantity in self.inventory.items():
                print(f'{instrument}: {quantity} шт.')
        print(f'Текущий баланс: {self.balance} нотакоинов.')


class Instrument:
    def __init__(self, name, buy_price, sell_price):
        self.name = name
        self.buy_price = buy_price
        self.sell_price = sell_price


class Piano(Instrument):
    def __init__(self, name, buy_price, sell_price, num_keys):
        super().__init__(name, buy_price, sell_price)
        self.num_keys = num_keys
    
    def __str__(self):
        return f'{self.name} ({self.num_keys} клавиш) | Цена покупки: {self.buy_price} | Цена продажи: {self.sell_price}.'


class Guitar(Instrument):
    def __init__(self, name, buy_price, sell_price, num_strings):
        super().__init__(name, buy_price, sell_price)
        self.num_strings = num_strings
    
    def __str__(self):
        return f'{self.name} ({self.num_strings} струн) | Цена покупки: {self.buy_price} | Цена продажи: {self.sell_price}.'


class Customer:
    def __init__(self, name, budget):
        self.name = name
        self.budget = budget
    
    def buy_instrument(self, store, instrument):
        if self.budget >= instrument.sell_price:
            if store.sell_instrument(instrument):
                self.budget -= instrument.sell_price
                print(f'Посетитель {self.name} купил {instrument.name} за {instrument.sell_price} нотакоинов.')
            else:
                print(f'Посетитель {self.name} хотел купить инструмент {instrument.name}, но его нет в наличии.')
        else:
            print(f'У посетителя {self.name} недостаточно денег для покупки {instrument.name}.')


class Game:
    def __init__(self):
        self.store = MusicStore('Pitch Perfect', 10000)
        self.instruments = [
            Guitar('Гитара', 800, 1000, 6),
            Piano('Пианино', 4000, 5000, 88)
        ]
        self.customers = [
            Customer('Петя', 15000),
            Customer('Иван', 5000),
            Customer('Мария', 13000),
            Customer('Алексей', 3000),
            Customer('Ольга', 12000),
        ]
        self.day = 1
        self.max_days = 7
    
    def run(self):
        print('MUSIC STORE SIMULATOR')
        while self.day <= self.max_days:
            print(f'\nДень {self.day}')
            if self.simulate_day() == 'exit':
                print('Игра завершена.')
                break
            self.day += 1
        else:
            print('\nПрошла неделя. Игра завершена.')
        self.final_results()
    
    def simulate_day(self):
        base_visitors = random.randint(0, 5)
        if self.store.promo_days > 0:
            extra_visitors = random.randint(1, 3)
            visitors = base_visitors + extra_visitors
            self.store.promo_days -= 1
            print(f'Реклама привлекла дополнительно {extra_visitors} посетителей.')
        else:
            visitors = base_visitors
        print(f'Сегодня магазин {self.store.name} посетило {visitors} человек.')
    
        for _ in range(visitors):
            customer = random.choice(self.customers)
            available_instruments = [instr for instr in self.instruments if instr.name in self.store.inventory and self.store.inventory[instr.name] > 0]
            if available_instruments:
                instrument = random.choice(self.instruments)
                customer.buy_instrument(self.store, instrument)
            else:
                print(f'Посетитель {customer.name} пришел в магазин, но ничего не купил, так как нет инструментов в наличии.')
        
        while True:
            action = self.user_action()
            if action == 'exit' or action == 'next_day':
                break

        if self.store.promo_days > 0:
            print(f'Эффект от публикации будет длиться еще {self.store.promo_days} днкй.')
        
        return action
    
    def user_action(self):
        print('\nВыберите действие:')
        print('1. Закупить инструменты')
        print('2. Начать рекламную кампанию')
        print('3. Просмотреть инвентарь')
        print('4. Завершить день')
        print('5. Выйти из игры')
        
        action = input('Ваш выбор: ')
        
        if action == '1':
            self.buy_instruments()
        elif action == '2':
            self.start_promotion()
        elif action == '3':
            self.store.display_inventory()
        elif action == '4':
            return 'next_day'
        elif action == '5':
            return 'exit'
        else:
            print('\nНекорректный выбор. Пожалуйста, выберите снова.')

    def buy_instruments(self):
        print('\nДоступные инструменты:')
        for i, instrument in enumerate(self.instruments, 1):
            print(f'{i}. {instrument}')
        
        choice = int(input('Выберите номер инструмента для покупки: ')) - 1
        if 0 <= choice < len(self.instruments):
            instrument = self.instruments[choice]
            quantity = int(input('Укажите количество: '))
            if quantity > 0:
                self.store.buy_instrument(instrument, quantity)
            else:
                print('Укажите положительное число.')
        else:
            print('\nНекорректный выбор инструмента.')
    
    def start_promotion(self):
        self.store.start_promotion(3000)
    
    def final_results(self):
        total_profit = self.store.balance - 10000 
        print('\nИТОГИ')
        print(f'Итоговый баланс: {self.store.balance} нотакоинов')
        print(f'Общая прибыль: {total_profit} нотакоинов')
        print(f'Продано инструментов: {self.store.total_sold}')


game = Game()
game.run()

MUSIC STORE SIMULATOR

День 1
Сегодня магазин Pitch Perfect посетило 1 человек.
Посетитель Петя пришел в магазин, но ничего не купил, так как нет инструментов в наличии.

Выберите действие:
1. Закупить инструменты
2. Начать рекламную кампанию
3. Просмотреть инвентарь
4. Завершить день
5. Выйти из игры

Доступные инструменты:
1. Гитара (6 струн) | Цена покупки: 800 | Цена продажи: 1000.
2. Пианино (88 клавиш) | Цена покупки: 4000 | Цена продажи: 5000.
Вы приобрели инструмент Гитара за 4000 нотакоинов.

Выберите действие:
1. Закупить инструменты
2. Начать рекламную кампанию
3. Просмотреть инвентарь
4. Завершить день
5. Выйти из игры
Вы опубликовали рекламный пост у инфлюенсера за 3000 нотакоинов. Это привлечет больше клиентов в следующие 3 дней.

Выберите действие:
1. Закупить инструменты
2. Начать рекламную кампанию
3. Просмотреть инвентарь
4. Завершить день
5. Выйти из игры
Эффект от публикации будет длиться еще 3 днкй.

День 2
Реклама привлекла дополнительно 3 посетителей.
Сегодня маг